In [226]:
import os,sys,json,re
import pandas as pd
import numpy as np

#### Notes:
The file directory in which the .rec files are located is named 'day' and needs to be within the working directory - or modified

#### Integrity Check of Data Files

In [251]:
''' Works if this notebook is in the working directory of the interpreter 
- otherwise, replace 'day' with the directory in which the rec files(and logs) are located'''
file_list = os.listdir('day') 
r0 = re.compile(r'^.*.rec$')
r1 = re.compile(r'^[0-9]{3}-(?P<day>day[0-9]+).rec$')
day = ''
flag = False
for file in file_list: #assume file is file name
    if r0.match(file): #if our .rec regex matches - we know it's a .rec file
        flag = True
        results = re.search(r1, file) #if no match results = None
        if not results:
            print("incorrectly formatted .rec")
            print(file)
            break
        else:
            if day == '':
                day = results.group('day') #gets day
            else:
                if results.group('day') != day:
                    print("incorrect day")
                    break
if not flag:
    print('No rec files found')

In [252]:
rec_list = [rec for rec in file_list if r0.match(rec)]
rec_list.sort()

#### DataFrame Creation

In [253]:
def getval(val):
    try:
        return val
    except:
        None

timestamps,uuids,pos_xs,pos_ys,neighbor_xs,neighbor_ys,phases,last_rec = [],[],[],[],[],[],[],[]
# last_rec signifies whether the timestamp belongs to the last recording of the session - used to accumulate the timestamps
nano = 0
for rec in rec_list:
    json_list = []
    with open('day/{}'.format(rec)) as f:
        for r in f.readlines():
            json_list.append(json.loads(r))
    for l in json_list:
        if l['message'] == '_SAVE_LIVEFRAME':
            for visual in l['data']['visuals']:
                if 'uuid' in visual and visual.get('type') == 'Mesh':
                    uuids.append(visual['uuid'])
                    timestamps.append(l['timestamp'] + nano)
                    pos_xs.append(float(visual['position']['x']))
                    pos_ys.append(float(visual['position']['y']))
                    neighbor_xs.append(float(visual['dashedLine']['x2']) if 'dashedLine' in visual and 'x2' in visual['dashedLine'] else None)
                    neighbor_ys.append(float(visual['dashedLine']['y2']) if 'dashedLine' in visual and 'y2' in visual['dashedLine'] else None)
                    phases.append(visual['dashedLine']['settings']['width'] 
                                  if 'dashedLine' in visual and 'settings' in visual['dashedLine'] and 'width' in visual['dashedLine']['settings'] else None)
                    last_rec.append(0)
    print(rec)
    if last_rec:
        last_rec[-1] = 1
        nano = timestamps[-1]
df_dict = {'uuids': uuids, 'timestamps':timestamps, 'pos_xs':pos_xs, 'pos_ys':pos_ys, 'neighbor_xs':neighbor_xs, 'neighbor_ys':neighbor_ys, 'phases':phases, 'last_rec': last_rec}
df = pd.DataFrame(df_dict)

001-day3.rec
002-day3.rec
003-day3.rec
004-day3.rec
005-day3.rec
006-day3.rec
007-day3.rec
008-day3.rec
009-day3.rec
010-day3.rec
011-day3.rec
012-day3.rec
013-day3.rec
014-day3.rec
015-day3.rec
016-day3.rec
017-day3.rec
018-day3.rec


In [254]:
df

,last_rec,neighbor_xs,neighbor_ys,phases,pos_xs,pos_ys,timestamps,uuids
0,0,-0.0404,0.2580,1.0,0.1490,-1.4500,41550280,A943B4A5-88AC-4E26-83C4-E2F78CD9D1E1
1,0,0.1490,-1.4500,15.0,-0.0404,0.2580,41550280,B43FDA85-832B-44D2-A8A6-5C0E75C06FE1
2,0,0.1820,0.2180,15.0,0.0134,-1.5200,143818794,A943B4A5-88AC-4E26-83C4-E2F78CD9D1E1
3,0,0.0134,-1.5200,15.0,0.1820,0.2180,143818794,B43FDA85-832B-44D2-A8A6-5C0E75C06FE1
4,0,0.5440,0.1150,15.0,-0.1220,-1.6000,244045742,A943B4A5-88AC-4E26-83C4-E2F78CD9D1E1
5,0,-0.1220,-1.6000,1.0,0.5440,0.1150,244045742,B43FDA85-832B-44D2-A8A6-5C0E75C06FE1
6,0,0.8020,0.0300,15.0,-0.2920,-1.6700,344189521,A943B4A5-88AC-4E26-83C4-E2F78CD9D1E1
7,0,-0.2920,-1.6700,1.0,0.8020,0.0300,344189521,B43FDA85-832B-44D2-A8A6-5C0E75C06FE1
8,0,1.0800,-0.1790,15.0,-0.4910,-1.7500,444909701,A943B4A5-88AC-4E26-83C4-E2F78CD9D1E1
9,0,-0.4910,-1.7500,1.0,1.0800,-0.1790,444909701,B43FDA85-832B-44D2-A8A6-5C0E75C06FE1


In [255]:
df['uuids'].unique() # Unique uuids - 4 uniques across the entire dataframe

array(['A943B4A5-88AC-4E26-83C4-E2F78CD9D1E1',
       'B43FDA85-832B-44D2-A8A6-5C0E75C06FE1',
       'B51BDF6B-A4F1-4284-BD1C-0AFCCDACAE99',
       '8E82AA58-ACBD-4B47-8C56-9A7E275102DC',
       'CE592D97-F593-4DEC-9D7E-03484B07CBA6',
       '177622F4-49B3-4B31-BE17-5FE72243F000',
       '03ACB8CE-058D-4B5E-81AC-C9D2C33C170F',
       '53A05400-71AB-4EFC-B1CD-50FD128721FB',
       'C46E3F94-2BF4-4C31-8837-5933AC54CA44',
       '6B41F94D-B0E4-45F5-822F-123373507D80',
       '1F0799E8-EE21-476D-8CF3-B1F74C904D6F'], dtype=object)

In [194]:
from bokeh.io import output_notebook
from bokeh.plotting import figure, output_file, show
output_notebook()

Loading BokehJS ...

Creates the timestamp and velocity series to visualize for one user

In [256]:
test_frame = df['uuids'].unique()[0]
time_diff = df[df['uuids'] == test_frame]['timestamps'].diff()
x_diff = df[df['uuids'] == test_frame]['pos_xs'].diff()
y_diff = df[df['uuids'] == test_frame]['pos_ys'].diff()

dist_diff = np.sqrt(x_diff**2 + y_diff**2)
velocity = dist_diff/time_diff

u1t = df[df['uuids'] == test_frame]['timestamps']

p = figure(x_axis_label='time', y_axis_label='velocity')
p.line(u1t, velocity)
show(p)

In [275]:
from bokeh.palettes import inferno
xs = []
ys = []
for u in df['uuids'].unique():
    test_frame = u
    time_diff = df[df['uuids'] == test_frame]['timestamps'].diff()
    x_diff = df[df['uuids'] == test_frame]['pos_xs'].diff()
    y_diff = df[df['uuids'] == test_frame]['pos_ys'].diff()

    dist_diff = np.sqrt(x_diff**2 + y_diff**2)
    velocity = dist_diff/time_diff

    u1t = df[df['uuids'] == test_frame]['timestamps']
    xs.append(u1t)
    ys.append(velocity)

p = figure(x_axis_label='time', y_axis_label='velocity')
p.multi_line(xs, ys, color=inferno(len(df['uuids'].unique())), alpha=[.5 for n in range(len(df['uuids'].unique()))])
show(p)